<center><b><font size="5">Deeply virtual Compton scattering at small xB and the access to the GPD H</font></b></center><br>
This is the jupyter/python code that reproduces some numbers and figures from paper by K. Kumericki and D. Mueller, <i>Deeply virtual Compton scattering at small xB and the access to the GPD H</i>, published in Nuclear Physics <b>B841</b> 1-58 (2010) [arXiv:<a href='https://arxiv.org/abs/0904.0458'>0904.0458</a>].

Figures for the published paper were produced by older Fortan version of the same code. They also contain NNLO numbers and curves, but Python gepard have only NLO implemented at the moment.

# Intro

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [2]:
import gepard as g
print(g.__version__)

0.9.9


In [3]:
DVCSpoints = g.data.dset[36].copy()
for id in range(37, 46):
    DVCSpoints += g.data.dset[id]

DISpoints = g.data.dset[201].copy()
for id in range(202, 213):
    DISpoints += g.data.dset[id]

# LO fits

## DIS

In [4]:
class MyTheory(g.PWNormGPD, g.MellinBarnesCFF, g.DIS, g.BMK):
    pass

In [5]:
th = MyTheory()

In [6]:
th.parameters.update({'ns': 0.15, 'al0s': 1., 'alps': 0.15, 'ms2': 1.,
                        'secs': 0., 'al0g': 1.1, 'alpg': 0.15, 'mg2': 0.7})

In [7]:
fitpoints = DISpoints
free_pars = ['ns', 'al0s', 'al0g']

In [8]:
f = g.MinuitFitter(DISpoints, th)
f.minuit.print_level = 1

f.fix_parameters('ALL')
f.release_parameters(*free_pars)
res = f.fit()

In [9]:
print('chisq/dof = {:.1f}/{}'.format(th.chisq(DISpoints), len(fitpoints)-len(free_pars)))

chisq/dof = 49.7/82


This is value from first column in Table 1  (49.7/82).

In [10]:
f.print_parameters()

ns    =    0.152 +- 0.005
al0s  =    1.158 +- 0.008
al0g  =    1.247 +- 0.014


More precise values:

In [11]:
for par in free_pars:
    print("'{}': {}".format(par, th.parameters[par]))

'ns': 0.15202785950397424
'al0s': 1.1575179117465697
'al0g': 1.2473613136604325


These values agree with Nsea, alpha^sea, and alpha^G columns of Table 2. (ns=0.152, al0s=1.158, al0g=1.247)

## DVCS, nl-SO3 PW model

In [12]:
fitpoints = DVCSpoints
free_pars = ['ms2', 'secs', 'secg']

In [13]:
f = g.MinuitFitter(fitpoints, th)
f.minuit.print_level = 1

In [14]:
%%time
f.fix_parameters('ALL')
f.release_parameters(*free_pars)
res = f.fit()

CPU times: user 24.7 s, sys: 3.03 ms, total: 24.7 s
Wall time: 24.8 s


In [15]:
print('chisq/dof = {:.1f}/{}'.format(th.chisq(fitpoints), len(fitpoints)-len(free_pars)))

chisq/dof = 96.5/98


This is within 1% of number in Table 1, (nl, dipole LO) row, DVCS column =  95.9/98

In [16]:
for par in free_pars:
    print("'{}': {}".format(par, th.parameters[par]))

'ms2': 0.4779843596710206
'secs': -0.15133465094004292
'secg': -0.8148297549360505


This agrees with Table 2, (nl, dipole LO) row:  ms=0.48; s^sea = -0.15; s^G = -0.81

# NLO fits

## DIS

In [17]:
th = MyTheory(p=1, scheme='msbar')                                                                                                                                                                                                                                                                                          
th.parameters.update({'ns': 0.15, 'al0s': 1., 'alps': 0.15, 'ms2': 1.,                                                                                                  
                        'secs': 0., 'al0g': 1.1, 'alpg': 0.15, 'mg2': 0.7})

In [18]:
fitpoints = DISpoints
free_pars = ['ns', 'al0s', 'al0g']

In [19]:
f = g.MinuitFitter(DISpoints, th)
f.minuit.print_level = 1

f.fix_parameters('ALL')
f.release_parameters(*free_pars)
res = f.fit()

In [20]:
print('chisq/dof = {:.1f}/{}'.format(th.chisq(DISpoints), len(fitpoints)-len(free_pars)))

chisq/dof = 71.6/82


This is value from first column in Table 3  (71.6/82).

In [21]:
for par in free_pars:
    print("'{}': {}".format(par, th.parameters[par]))

'ns': 0.16780018553930706
'al0s': 1.1283626215016564
'al0g': 1.0990616571425693


These values agree with Nsea, alpha^sea, and alpha^G columns of Table 4. (ns=0.168, al0s=1.128, al0g=1.099)

## DVCS, nl-SO3 PW model

In [22]:
fitpoints = DVCSpoints
free_pars = ['ms2', 'secs', 'secg']

In [23]:
%%time
f = g.MinuitFitter(fitpoints, th)
f.minuit.print_level = 1

f.fix_parameters('ALL')
f.release_parameters(*free_pars)
res = f.fit()

CPU times: user 42.9 s, sys: 3.21 ms, total: 42.9 s
Wall time: 42.9 s


In [24]:
print('chisq/dof = {:.1f}/{}'.format(th.chisq(fitpoints), len(fitpoints)-len(free_pars)))

chisq/dof = 104.8/98


This is close to number in Table 3, (nl, NLO-MSBAR) row, DVCS column =  102/98

In [25]:
for par in free_pars:
    print("'{}': {}".format(par, th.parameters[par]))

'ms2': 0.5866625801160781
'secs': 0.03446689910295511
'secg': -0.007181242445842039


This is reasonably close to numbers in Table 4, (nl, NLO-MSBAR) row:  ms=0.59; s^sea = 0.04; s^G = -0.02.

Note that singlet MSBAR evolution in the code used for published paper had a small bug in the part nondiagonal in conformal moments, so some small numerical discrepancies are expected.

# NLO fits - CSBAR scheme

## DIS

For DIS MSBAR=CSBAR, but we have to run this anyway to set parameters for DVCS

In [26]:
th = MyTheory(p=1, scheme='csbar')                                                                                                                                                                                                                                                                                           
th.parameters.update({'ns': 0.15, 'al0s': 1., 'alps': 0.15, 'ms2': 1.,                                                                                                  
                        'secs': 0., 'al0g': 1.1, 'alpg': 0.15, 'mg2': 0.7})

In [27]:
fitpoints = DISpoints
free_pars = ['ns', 'al0s', 'al0g']

In [28]:
f = g.MinuitFitter(DISpoints, th)
f.minuit.print_level = 1

f.fix_parameters('ALL')
f.release_parameters(*free_pars)
res = f.fit()

In [29]:
print('chisq/dof = {:.1f}/{}'.format(th.chisq(DISpoints), len(fitpoints)-len(free_pars)))

chisq/dof = 71.6/82


This is the value from the first column in Table 3  (71.6/82).

In [30]:
for par in free_pars:
    print("'{}': {}".format(par, th.parameters[par]))

'ns': 0.16780018553930706
'al0s': 1.1283626215016564
'al0g': 1.0990616571425693


These values agree with Nsea, alpha^sea, and alpha^G columns of Table 4. (ns=0.168, al0s=1.128, al0g=1.099)

## DVCS, nl-SO3 PW model

In [31]:
fitpoints = DVCSpoints
free_pars = ['ms2', 'secs', 'secg']

In [32]:
%%time
f = g.MinuitFitter(fitpoints, th)
f.minuit.print_level = 1

f.fix_parameters('ALL')
f.release_parameters(*free_pars)
res = f.fit()

CPU times: user 11.5 s, sys: 3.31 ms, total: 11.5 s
Wall time: 11.5 s


In [33]:
print('chisq/dof = {:.1f}/{}'.format(th.chisq(fitpoints), len(fitpoints)-len(free_pars)))

chisq/dof = 105.8/98


This is close enough to the number in Table 3, (nl, NLO-CSBAR) row, DVCS column =  104./98

In [34]:
for par in free_pars:
    print("'{}': {}".format(par, th.parameters[par]))

'ms2': 0.5778597077034826
'secs': -0.006509961901584814
'secg': -0.016135260400876653


This closely agrees with Table 4, (nl, NLO-CSBAR) row:  ms=0.58; s^sea = -0.01; s^G = -0.01.